# 🌊 Flood Extent Analysis using Sentinel-1 SAR (Google Earth Engine)

**Author:** Flood Analysis Pipeline  
**Date:** February 2026  
**Case Study:** 2022 Pakistan Catastrophic Floods – Sindh Province

---

## Overview

This notebook demonstrates a complete end-to-end flood mapping pipeline using **Sentinel-1 Synthetic Aperture Radar (SAR)** imagery accessed via **Google Earth Engine (GEE)**.

### Why Sentinel-1 SAR?
Unlike optical satellites (Landsat, Sentinel-2), SAR sensors can:
- ✅ **Penetrate clouds** – critical because floods almost always occur under heavy cloud cover
- ✅ **Image day and night** – active radar illumination
- ✅ **Detect standing water** precisely – smooth water surfaces cause very low radar backscatter

### Methodology
1. Load Sentinel-1 GRD images before and after the flood event
2. Apply speckle filtering (reduce SAR noise)
3. Calculate backscatter difference (pre − post)
4. Apply thresholds to isolate flooded areas
5. Remove permanent water bodies
6. Calculate affected area statistics
7. Visualize and export results

## Section 1: Setup and Authentication

In [ ]:
# Install required packages (run only if needed)
# !pip install earthengine-api geemap folium matplotlib pandas numpy

import ee
import geemap
import geemap.colormaps as cm
import folium
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
from datetime import datetime

print('✅ All libraries imported successfully')

In [ ]:
# Authenticate and Initialize Earth Engine
# First-time users: this will open a browser window to log in.
try:
    ee.Initialize()
    print('✅ Earth Engine initialized successfully (using cached credentials)')
except Exception as e:
    print('Authenticating... a browser window will open.')
    ee.Authenticate()
    ee.Initialize()
    print('✅ Earth Engine initialized successfully after authentication')

## Section 2: Define Study Area and Timeframes

We focus on the **Sindh Province, Pakistan** – one of the worst-hit regions during the historic 2022 monsoon floods that submerged roughly one-third of Pakistan.

In [ ]:
# ------------------------------------------------------------------
# CONFIGURATION – Change these to study a different area or event
# ------------------------------------------------------------------

# Area of Interest: Sindh Province, Pakistan
AOI = ee.Geometry.Polygon(
    [[[67.5, 25.5],
      [67.5, 27.5],
      [69.5, 27.5],
      [69.5, 25.5]]]
)

# Map center for visualization
CENTER_LAT = 26.5
CENTER_LON = 68.5
ZOOM = 8

# Timeframes
PRE_FLOOD_START  = '2022-05-01'
PRE_FLOOD_END    = '2022-06-30'
POST_FLOOD_START = '2022-08-15'
POST_FLOOD_END   = '2022-09-15'

# SAR Thresholds (dB)
FLOOD_THRESHOLD = -3    # Minimum difference (post - pre) to mark as flooded
WATER_THRESHOLD = -16   # Maximum VH backscatter of water surfaces

print(f'📍 Study Area: Sindh Province, Pakistan ({CENTER_LAT}°N, {CENTER_LON}°E)')
print(f'📅 Pre-flood period:  {PRE_FLOOD_START} → {PRE_FLOOD_END}')
print(f'📅 Post-flood period: {POST_FLOOD_START} → {POST_FLOOD_END}')

## Section 3: Load and Preprocess Sentinel-1 SAR Imagery

In [ ]:
# Load Sentinel-1 GRD collection
s1 = (ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterBounds(AOI)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select(['VH', 'VV'])
)

# Separate pre- and post-flood images
pre_collection  = s1.filterDate(PRE_FLOOD_START, PRE_FLOOD_END)
post_collection = s1.filterDate(POST_FLOOD_START, POST_FLOOD_END)

print(f'Pre-flood  image count:  {pre_collection.size().getInfo()}')
print(f'Post-flood image count:  {post_collection.size().getInfo()}')

In [ ]:
# Create mosaics clipped to AOI
pre_mosaic  = pre_collection.mosaic().clip(AOI)
post_mosaic = post_collection.mosaic().clip(AOI)

# Apply speckle filter (focal mean)
SMOOTH_RADIUS = 50  # metres

pre_vh  = pre_mosaic.select('VH').focal_mean(SMOOTH_RADIUS, 'circle', 'meters')
post_vh = post_mosaic.select('VH').focal_mean(SMOOTH_RADIUS, 'circle', 'meters')

print('✅ Speckle filtering applied (50m focal mean)')

## Section 4: Flood Detection via Change Detection

In [ ]:
# Calculate backscatter difference (post minus pre)
# A large NEGATIVE difference indicates a significant drop in backscatter → flooding
difference = post_vh.subtract(pre_vh)

# Threshold-based flood extraction
# Pixels must satisfy BOTH conditions:
#   1. Backscatter dropped significantly (difference < FLOOD_THRESHOLD)
#   2. Post-flood backscatter is very low – characteristic of open water (post_vh < WATER_THRESHOLD)
flooded_raw = (difference.lt(FLOOD_THRESHOLD)
                          .And(post_vh.lt(WATER_THRESHOLD)))

# Load JRC Global Surface Water to mask out permanent water bodies
jrc = ee.Image('JRC/GSW1_4/GlobalSurfaceWater')
permanent_water = jrc.select('seasonality').gte(10)  # water present ≥10 months/year

# Remove permanent water from flood map
flood_extent = flooded_raw.where(permanent_water, 0)
flood_masked = flood_extent.updateMask(flood_extent)

print('✅ Flood extent extracted and permanent water removed')

## Section 5: Area Statistics

In [ ]:
# Calculate total flooded area in hectares
flooded_area_img = flood_masked.multiply(ee.Image.pixelArea()).divide(10000)  # m² to hectares

stats = flooded_area_img.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=AOI,
    scale=500,          # use 500m resolution for computation speed
    maxPixels=1e10,
    bestEffort=True
).getInfo()

flooded_ha = stats.get('VH', 0)
flooded_km2 = flooded_ha / 100

# Also estimate AOI total area
aoi_area_ha = AOI.area(maxError=1).getInfo() / 10000
pct_flooded = (flooded_ha / aoi_area_ha) * 100

print('=' * 50)
print('📊 FLOOD STATISTICS')
print('=' * 50)
print(f'  Estimated Flooded Area:  {flooded_ha:>12,.0f} ha')
print(f'  Estimated Flooded Area:  {flooded_km2:>12,.0f} km²')
print(f'  AOI Total Area:          {aoi_area_ha:>12,.0f} ha')
print(f'  % of AOI Flooded:        {pct_flooded:>11.1f} %')
print('=' * 50)

## Section 6: Interactive Map Visualization

In [ ]:
sar_vis  = {'min': -25, 'max': -5, 'palette': ['black', 'white']}
diff_vis = {'min': -10, 'max': 2,  'palette': ['blue', 'white', 'red']}

Map = geemap.Map(center=[CENTER_LAT, CENTER_LON], zoom=ZOOM)
Map.add_basemap('HYBRID')

Map.addLayer(pre_vh,      sar_vis,               'Pre-Flood SAR (VH)',  False)
Map.addLayer(post_vh,     sar_vis,               'Post-Flood SAR (VH)', False)
Map.addLayer(difference,  diff_vis,              'SAR Difference',      False)
Map.addLayer(flood_masked, {'palette': ['00CFFF']}, '🌊 Flood Extent',    True)
Map.addLayer(permanent_water.updateMask(permanent_water),
             {'palette': ['0000FF']}, 'JRC Permanent Water',  False)

Map.addLayerControl()
Map

## Section 7: Statistical Charts

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.patch.set_facecolor('#0E1117')

for ax in axes:
    ax.set_facecolor('#1A1F2E')
    ax.tick_params(colors='white')
    ax.spines[:].set_color('#2A3050')

# --- Chart 1: Area comparison bar chart ---
labels = ['Total AOI Area', 'Flooded Area']
values = [aoi_area_ha / 1e6, flooded_ha / 1e6]  # Convert to million ha
colors = ['#2A3050', '#00CFFF']
bars = axes[0].bar(labels, values, color=colors, edgecolor='white', width=0.5)
axes[0].set_title('Area Comparison (Million ha)', color='white', fontsize=13, pad=15)
axes[0].set_ylabel('Area (Million ha)', color='white')
for bar, val in zip(bars, values):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
                 f'{val:.2f}M', ha='center', color='white', fontweight='bold')

# --- Chart 2: Pie chart of flooded vs non-flooded ---
pie_vals  = [flooded_ha, aoi_area_ha - flooded_ha]
pie_lbls  = [f'Flooded\n({pct_flooded:.1f}%)', 'Non-Flooded']
pie_cols  = ['#00CFFF', '#2A3050']
axes[1].pie(pie_vals, labels=pie_lbls, colors=pie_cols,
            startangle=90, textprops={'color': 'white'},
            wedgeprops={'edgecolor': 'white', 'linewidth': 1.2},
            autopct='%1.1f%%', pctdistance=0.7)
axes[1].set_title('Flooded vs Non-Flooded Land', color='white', fontsize=13, pad=15)

# --- Chart 3: Simulation of historical flood progression (illustrative) ---
months = ['Jun', 'Jul', 'Aug', 'Sep', 'Oct']
flood_pct_sim = [0.5, 2.1, 8.4, pct_flooded, 3.2]
axes[2].plot(months, flood_pct_sim, color='#00CFFF', linewidth=2.5, marker='o',
             markerfacecolor='white', markeredgecolor='#00CFFF', markersize=7)
axes[2].fill_between(months, flood_pct_sim, alpha=0.2, color='#00CFFF')
axes[2].set_title('Flood Progression 2022 (Simulated %)', color='white', fontsize=13, pad=15)
axes[2].set_ylabel('% Area Flooded', color='white')
axes[2].set_xlabel('Month (2022)', color='white')

plt.suptitle('🌊 2022 Pakistan Flood Analysis – Sindh Province',
             color='white', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('flood_statistics_chart.png', dpi=150, bbox_inches='tight',
            facecolor='#0E1117')
plt.show()
print('✅ Chart saved as flood_statistics_chart.png')

## Section 8: Export Flood Extent GeoTIFF

In [ ]:
# Optional: Export the flood extent to Google Drive as GeoTIFF
# Uncomment and run to trigger an export task

# task = ee.batch.Export.image.toDrive(
#     image=flood_masked,
#     description='Pakistan_Flood_Extent_2022',
#     folder='FloodAnalysis',
#     fileNamePrefix='pakistan_flood_2022_sindhprovince',
#     region=AOI,
#     scale=30,
#     crs='EPSG:4326',
#     maxPixels=1e10
# )
# task.start()
# print(f'Export task started: {task.id}')
# print('Check status at: https://code.earthengine.google.com/tasks')

print('ℹ️  To export the flood extent GeoTIFF, uncomment the code above and run this cell.')
print('   The export will appear in your Google Drive under the "FloodAnalysis" folder.')

## Summary

| Item | Value |
|------|-------|
| **Dataset** | Sentinel-1 GRD (VH polarization) |
| **Method** | SAR backscatter change detection + thresholding |
| **Event** | 2022 Pakistan Monsoon Floods |
| **Region** | Sindh Province |
| **Permanent water mask** | JRC Global Surface Water v1.4 |

### References
- ESA Sentinel-1 Mission: https://sentinel.esa.int/web/sentinel/missions/sentinel-1
- UN SPIDER SAR flood mapping guide: https://un-spider.org/advisory-support/recommended-practices/recommended-practice-flood-mapping
- JRC Global Surface Water: Pekel et al. (2016), Nature